In [1]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
#     var[var > 1000] = np.nan
#     var[var < -1000] = np.nan
    var = var[:, 29:] 
    return var

def get_data_from_mat_for_calcu(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:]
    return var.T

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

data_surface = Data_from_nc(\
                            '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/' + \
                            'surfdata_irrigation_method.nc'\
                           )   #load the data

ar6_region = get_data_from_mat_for_calcu('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/ar6_region.mat','ar6_region')
irr_diff = get_data_from_mat_for_calcu('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/irr_diff_out.mat', 'irr_diff_out')

data_irrigation_method = data_surface.load_variable('irrigation_method')

data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]






def get_data_for_year(metric, period, exp, ens):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_1.mat', metric)
    data_out = np.array(data_out)
    return data_out

def get_data_for_year_delta_irr(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_1.mat', metric)
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

AREA = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/input_data/AREA.mat', 'AREA') # Area for CLM_data
AREA = AREA.T

def get_hour_km2_years(data_period, qctl):
    data_time_one_year = np.array(range(2920))
    
    data_period_year = data_period[0 : 2920] 
    data_time_one_year = np.array(range(data_period_year.shape[0]))
    data_xr_new = xr.DataArray(data_period_year, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    
    return hour_km_2
    
def plot_line(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=12, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 1, marker = marker, markersize=1.2, alpha=0.8)
    ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right')
    plt.title(title1, loc='left')
    plt.ylabel(ylabel, fontsize = 14)
    plt.xlabel(xlabel, fontsize = 14)
    plt.yticks(fontsize = 12)
    plt.xticks(fontsize=12)
    plt.legend()
    

def all_delta_irr(metric, perc, unit, variable_to_show, irr_low, irr_high):
    data_NOI01_1901_1930 = get_data_for_year_delta_irr(metric, '1901', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI02_1901_1930 = get_data_for_year_delta_irr(metric, '1901', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    for period in range(1902, 1931):
        data_NOI01_year = get_data_for_year_delta_irr(metric, str(period), 'NOI', '01', irr_low, irr_high, irr_diff)
        data_NOI01_1901_1930 = np.concatenate((data_NOI01_1901_1930, data_NOI01_year), axis = 0)
        data_NOI02_year = get_data_for_year_delta_irr(metric, str(period), 'NOI', '02', irr_low, irr_high, irr_diff)
        data_NOI02_1901_1930 = np.concatenate((data_NOI02_1901_1930, data_NOI02_year), axis = 0)

    
    data_NOI_1901_1930 = np.concatenate((data_NOI01_1901_1930,
                                         data_NOI02_1901_1930
                                         ), axis = 0)
    del data_NOI01_1901_1930, data_NOI02_1901_1930
    gc.collect()
    
    data_time = np.array(range(data_NOI_1901_1930.shape[0]))
    data_xr_ref = xr.DataArray(data_NOI_1901_1930, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qctl=data_xr_ref.quantile(perc,dim='time')
    
    hour_km_2_IRR01 = np.zeros(114) 
    hour_km_2_NOI01 = np.zeros(114)
    
    hour_km_2_IRR02 = np.zeros(114) 
    hour_km_2_NOI02 = np.zeros(114)
    

    
    for period in range(1901, 2015):
        data_IRR01_year = get_data_for_year_delta_irr(metric, str(period), 'IRR', '01', irr_low, irr_high, irr_diff)
        data_IRR02_year = get_data_for_year_delta_irr(metric, str(period), 'IRR', '02', irr_low, irr_high, irr_diff)
        
        
        data_NOI01_year = get_data_for_year_delta_irr(metric, str(period), 'NOI', '01', irr_low, irr_high, irr_diff)
        data_NOI02_year = get_data_for_year_delta_irr(metric, str(period), 'NOI', '02', irr_low, irr_high, irr_diff)
        
        hour_km_2_IRR01[period - 1901] = get_hour_km2_years(data_IRR01_year, qctl)
        hour_km_2_NOI01[period - 1901] = get_hour_km2_years(data_NOI01_year, qctl)
        
        hour_km_2_IRR02[period - 1901] = get_hour_km2_years(data_IRR02_year, qctl)
        hour_km_2_NOI02[period - 1901] = get_hour_km2_years(data_NOI02_year, qctl)

        
    data_IRR = np.vstack((hour_km_2_IRR01,
                               hour_km_2_IRR02))
    data_NOI = np.vstack((hour_km_2_NOI01,
                               hour_km_2_NOI02))

    return data_IRR, data_NOI
    


In [2]:
def all(var, percentile, unit, var_to_show, irr_low, irr_high, name, index):
    data_IRR, data_NOI=all_delta_irr(var, percentile, unit, var_to_show, irr_low, irr_high)
    
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/data_IRR_" + \
                     var + '_' + str(percentile) + '_' + str(irr_low) + '_' + str(irr_high) + '.mat', {'data_IRR': data_IRR})
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/data_NOI_" + \
                     var + '_' + str(percentile) + '_' + str(irr_low) + '_' + str(irr_high) + '.mat', {'data_NOI': data_NOI})
    data_IRR_75 = np.percentile(data_IRR, 75, axis = 0)
    data_IRR_50 = np.percentile(data_IRR, 50, axis = 0)
    data_IRR_25 = np.percentile(data_IRR, 25, axis = 0)

    data_NOI_75 = np.percentile(data_NOI, 75, axis = 0)
    data_NOI_50 = np.percentile(data_NOI, 50, axis = 0)
    data_NOI_25 = np.percentile(data_NOI, 25, axis = 0)
    
    fig = plt.figure(figsize=(12, 6),dpi=300)
    fig.subplots_adjust(hspace=0.4, wspace=0.2, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)
    ax1 = plt.subplot(224, frameon=True)
    plt.title(name, loc='right')
    ax1.spines["top"].set_visible(False)
    ax1.spines["right"].set_visible(False)
    plot_line(ax1, index, data_NOI_50, data_NOI_25, data_NOI_75, '', '', '1901irr', 'r', r'$\Delta$(km^3*hours)', 'year', '')
    plot_line(ax1, index, data_IRR_50, data_IRR_25, data_IRR_75, '', '', 'tranirr', 'b', r'$\Delta$(km^3*hours)', 'year', '')


In [ ]:
all('T_w', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 0.01, 20, '', 'a ' + 'SAS')
all('T_w', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 20, 30, '', 'a ' + 'SAS')
all('T_w', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 30, 40, '', 'a ' + 'SAS')
all('T_w', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 40, 100, '', 'a ' + 'SAS')
all('T_w', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 0.01, 20, '', 'a ' + 'SAS')
all('T_w', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 20, 30, '', 'a ' + 'SAS')
all('T_w', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 30, 40, '', 'a ' + 'SAS')
all('T_w', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 40, 100, '', 'a ' + 'SAS')
all('T_w', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 0.01, 20, '', 'a ' + 'SAS')
all('T_w', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 20, 30, '', 'a ' + 'SAS')
all('T_w', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 30, 40, '', 'a ' + 'SAS')
all('T_w', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 40, 100, '', 'a ' + 'SAS')